In [ ]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

# TOTAL FACTURAS 

In [5]:
import xmlrpc.client
from datetime import date, timedelta, datetime
import pandas as pd

# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
weeks = 52

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["product_id", "invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df_merged = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df_merged = df_merged.drop(columns=["invoice_id_num", "product_id_num"])


df_merged['origin'] = df_merged['origin'].astype('string')

# Guardar
df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas con información de factura: {len(df_merged)}")

#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos

Facturas encontradas: 51134
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 80000 registros de líneas...
Descargados 100000 registros de líneas...
Descargados 120000 registros de líneas...
Descargados 140000 registros de líneas...
Descargados 160000 registros de líneas...
Descargados 180000 registros de líneas...
Descargados 199327 registros de líneas...
Total de líneas descargadas: 199327
Total de líneas descargadas con información de factura: 199327


In [6]:
print(f"Fecha primera factura",df_merged["date_invoice"].min())
print(f"Fecha ultima factura",df_merged["date_invoice"].max())

Fecha primera factura 2024-10-02 12:25:36
Fecha ultima factura 2025-10-01 14:07:04


In [7]:
pd.set_option("display.max_colwidth", None)  # muestra toda la celda completa
df_merged.sample(10)

,origin,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
150339,False,61500.0,1446441,1.0,[DAB04926025] DA4926 FILTRO AIRE INTERNO IHC 4300-7400 (025 DA4926),PV2E36306 Mostrador Calle 6 T3/21012,2025-01-10 16:26:23,PV2E36306,393634,SUCURSAL CALLE 6,ANDRES FELIPE ORDOÑEZ GREGORY,SUCURSALES / Ventas Mostrador Cll6,37668,[CC80390526] MARINO RIVERA RIVEROS
61497,BD2\OUT\103762,40100.0,1551850,1.0,[DCS10197189] GS197A1 FILTRO COMBUSTIBLE SEPARADOR - CUMMINS - FORD (189 GS197A1),FCL61503,2025-06-14 16:42:32,FCL61503,421478,SUCURSAL CALLE 6,Camila Arguello,SUCURSALES / Ventas Mostrador Cll6,3929,[CO8220001381] INGECON S.A.
54794,False,252200.0,1559261,1.0,[BAP02988125] DA2988 FILTRO AIRE CHANNEL FLOW FREIGHTLINER (125 CA4700),PV4E15398 Mostrador Norte T1/17466,2025-06-27 16:27:37,PV4E15398,423562,SUCURSAL NORTE,WILMER MULATO,SUCURSALES / Ventas Mostrador Norte,35744,[CC7185383] WILLIAM REYES REYES
83513,BD7/OUT/14952,134400.0,1525687,6.0,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITACHI (CARCASA GX2772A) (025 DA2772)",FBUC5987,2025-05-07 15:19:33,FBUC5987,414493,SUCURSAL BUCARAMANGA,SANDRA RAMIREZ,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,43397,[CO9016553502] INVERSIONES LEON Y GUERRERO S.A.S.
125198,BD6/OUT/43874,102000.0,1476009,4.0,[DAB02612025] DA2612 FILTRO AIRE 1_ TOYOTA MONTACARGAS. (025 DA2612),FBAQ12691,2025-02-20 18:37:27,FBAQ12691,401421,SUCURSAL BARRANQUILLA,JAIME ALBERTO CAÑON RAMIREZ,SUCURSALES / BARRANQUILLA / EXTERNOS,3038,[CO8001171122] DIGABE DEL CARIBE Y COMPAÑIA SAS
160783,False,55800.0,1433104,1.0,[BCS00531125] GS531 FILTRO COMBUSTIBLE SEPARADOR THERMO KING (125 BF9894),PV9E666 Mostrador Cali T1/0827,2024-12-13 13:23:26,PV9E666,390366,SUCURSAL CALI,CLAUDIA ORTIZ,SUCURSALES / CALI / MOSTRADOR CALI CONTADO,38887,[CO10949076346] JORGE LUIS CRUZ ACOSTA
24389,False,156400.0,1596858,1.0,[DCS00564137] GS564 FILTRO COMBUSTIBLE SEPARADOR MERCEDES BENZ (137 R160T),PV7E7972 Mostrador Bucaramanga T1/7899,2025-08-21 19:29:55,PV7E7972,433124,SUCURSAL BUCARAMANGA,EDISSON MIGUEL CIFUENTES OSORIO,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,22625,[CO9005247753] ALL MACHINES S.A.S
92050,BD1\OUT\58278,46500.0,1515974,1.0,[BLE00668125] G668 FILTRO ACEITE FREIGTHLINER BUSSINES CLASS M2 (125 P7188),FCOT24088,2025-04-22 17:44:03,FCOT24088,411795,PRINCIPAL COTA,JUAN CARLOS BETANCOURT GUTIERREZ,GERENCIA,3629,[CO8300388058] REPRESENTACIONES OIL FILTERS S.A.
58823,False,46300.0,1554741,1.0,[BCS00252125] GS252 FILTRO COMBUSTIBLE BOBCAT CON DRENAJE MANUAL (125 BF1257),PV2E40822 Mostrador Calle 6 T2/20540,2025-06-19 16:39:11,PV2E40822,422239,SUCURSAL CALLE 6,JAIME ANDRES CABALLERO CABALLERO,SUCURSALES / Ventas Mostrador Cll6,13300,[CO9006457659] TRANSERVICIOS Y CONSTRUCCIONES S.A.S.
3201,False,93800.0,1621517,2.0,[DAB04688025] DA4688 FILTRO AIRE SEGURIDAD_ CATERPILLAR 6I2504 (025 DA4688),PV7E8285 Mostrador Bucaramanga T2/2566,2025-09-24 20:53:39,PV7E8285,439657,SUCURSAL BUCARAMANGA,SANDRA RAMIREZ,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,27557,[CC9162843] VICTOR ARMANDO FONSECA FONSECA


# COSTO DEL PRODUCTO

In [ ]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# ANALISIS COSTOS

In [12]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df.to_csv("real_ema.csv")

#timpo aprox  1 minutos

# VENTAS PERDIDAS

In [ ]:
import xmlrpc.client
from datetime import date
import pandas as pd
import warnings

# Ignorar la advertencia de `SettingWithCopyWarning` temporalmente
# Esta es una buena práctica para evitar que el código se detenga
# ===============================
# 1. Conexión con Odoo
# ===============================

# Variables de conexión
ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Leer ventas perdidas
# ===============================

# Definir la fecha de inicio para la consulta (desde 2024-12-01)
FECHA_INICIO = "2024-12-01"

# Buscar los IDs de las ventas perdidas
lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["almacenamiento_tipo", "=", "agotado"]]]
)

# Leer los datos de las ventas perdidas usando los IDs
lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

# Convertir la lista de diccionarios a un DataFrame de Pandas
df = pd.DataFrame(lost_sales_data)

# ===============================
# 3. Normalizar columnas con tuplas (many2one)
# ===============================

# Usar el método .str para extraer el primer y segundo elemento de las tuplas de forma eficiente
for col in ["store_id", "cliente_id"]:
    # Crea una nueva columna para el nombre (e.g., store_name)
    df[col.replace('_id', '_name')] = df[col].apply(lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None)
    # Crea una nueva columna para el ID (e.g., store_id_num)
    df[col + "_num"] = df[col].apply(lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None)

# La columna 'product_ref' ya es una cadena y no necesita normalización
# Las columnas product_id, warehouse_id y warehouse_name no se incluyeron en el READ, así que no es necesario procesarlas

# ===============================
# 4. Limpiar el DataFrame
# ===============================

# Eliminar las columnas que no son necesarias para el análisis final
# La columna 'id' se puede eliminar ya que no es relevante
# Ahora que extrajimos el nombre, podemos eliminar las columnas originales de tuplas
df = df.drop(columns=["id", "store_id"], errors='ignore')

# Renombrar las columnas para mayor claridad
#df = df.rename(columns={
 #   "product_ref": "referencia_producto",
  #  "store_name": "nombre_tienda",
   # "cliente_name": "nombre_cliente",
#})



# ===============================
# 5. Exportar a Parquet
# ===============================
try:
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="pyarrow", compression="snappy")
    print("✅ Exportación exitosa con pyarrow")
except ImportError as e:
    print("⚠️ Módulo pyarrow no encontrado, probando con fastparquet:", e)
    try:
        df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="fastparquet", compression="snappy")
        print("✅ Exportación exitosa con fastparquet")
    except ImportError as e:
        print("❌ Módulo fastparquet tampoco encontrado. Por favor, instala uno de los dos con 'pip install pyarrow' o 'pip install fastparquet'.")

print("\n")
print(f"El proceso ha terminado. Se extrajeron {len(df)} registros de ventas perdidas.")
print("Las columnas del DataFrame final son:", list(df.columns))


    ##TARDA MAS O MENOS 1 MINUTOS TODO 2025


✅ Exportación exitosa con pyarrow


El proceso ha terminado. Se extrajeron 58285 registros de ventas perdidas.
Las columnas del DataFrame final son: ['cantidad_reservada', 'product_ref', 'cantidad', 'almacenamiento_tipo', 'fecha', 'cantidad_existencia', 'origen', 'store_name', 'store_id_num', 'cliente_name', 'cliente_id_num']


# MOTOR BUSQUEDA

In [3]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponible
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "res.partner", "fields_get", 
    []
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:25} | {info.get('type')} | {info.get('string')}")



claim_count               | integer | # Claims
issue_count               | integer | # Issues
meeting_count             | integer | # Meetings
supplier_invoice_count    | integer | # Supplier Invoices
task_count                | integer | # Tasks
purchase_order_count      | integer | # of Purchase Order
sale_order_count          | integer | # of Sales Order
property_account_payable  | many2one | Account Payable
property_account_prepayable | many2one | Account Payable (Prepayment)
property_account_receivable | many2one | Account Receivable
property_account_prereceivable | many2one | Account Receivable (Prepayment)
active                    | boolean | Active
activity_economic_id      | many2one | Actividad economica
type                      | selection | Address Type
payment_amount_due        | float | Amount Due
payment_amount_overdue    | float | Amount Overdue
answers_ids               | many2many | Answers
is_autorizado             | boolean | Autorizado
bank_ids                  |

In [ ]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")

#aprox2min

In [ ]:
r =resultado.sort_values(["salario_neto"],ascending=False)
r = r[r["periodo"]=="2025-09"]
r.head(30)